In [219]:
# pip install sodapy

In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import collections;
import re;
from time import time
from collections import defaultdict

# Read Data Using API

In [2]:
# client = Socrata("data.sfgov.org", None)
# results = client.get("cuks-n6tp", limit = 2191368)
# data = client.get("cuks-n6tp", limit = 3000000)
# data_df = pd.DataFrame.from_records(data)

# Read data from CSV

In [3]:
import pandas as pd

data_df = pd.read_csv("./data/crimedata.csv");

In [4]:
# print(type(input))
data_df = data_df.drop(data_df.columns[0], axis=1)

In [5]:
print(type(data_df))
print(data_df.shape)

<class 'pandas.core.frame.DataFrame'>
(2195020, 13)


In [6]:
print(data_df[:2])

                   address        category                     date dayofweek  \
0    KISSLING ST / 11TH ST  SUSPICIOUS OCC  2015-06-06T00:00:00.000  Saturday   
1  JACKSON ST / WEBSTER ST    NON-CRIMINAL  2015-09-10T00:00:00.000  Thursday   

                       descript  incidntnum  \
0       INVESTIGATIVE DETENTION   150494743   
1  AIDED CASE, MENTAL DISTURBED   150795745   

                                            location pddistrict  \
0  {u'type': u'Point', u'coordinates': [-122.4150...   SOUTHERN   
1  {u'type': u'Point', u'coordinates': [-122.4329...   NORTHERN   

             pdid resolution   time           x          y  
0  15049474364085       NONE  15:14 -122.415003  37.772650  
1  15079574564020       NONE  16:40 -122.432920  37.792679  


In [7]:
# data_df.sort_values(by=['date', 'time'])
# data_df.to_csv('./data/crimedata.csv', sep= ",", encoding="utf8")

In [8]:
print(data_df.shape)

(2195020, 13)


In [9]:
for col in data_df.columns:
    print(col)

address
category
date
dayofweek
descript
incidntnum
location
pddistrict
pdid
resolution
time
x
y


In [10]:
def convert_date_to_day(dt):
    result = re.findall(r'\d{4}-(\d{2})-(\d{2})T00:00:00.000',dt)
    return result[0][1]
   
def convert_date_to_month(dt):
    result = re.findall(r'\d{4}-(\d{2})-(\d{2})T00:00:00.000',dt)
    return result[0][0]

def convert_time_to_hour(tm):
    result = re.findall(r'(\d{2}):\d{2}',tm)
    return result[0]

# def convert_date_to_year(dt):
#     result = re.findall(r'\d{4}-(\d{2})-(\d{2})T00:00:00.000',dt)
#     return result[0][0]

In [11]:
data_df = data_df.dropna(how='any',axis=0)
print(data_df[:1])

                 address        category                     date dayofweek  \
0  KISSLING ST / 11TH ST  SUSPICIOUS OCC  2015-06-06T00:00:00.000  Saturday   

                  descript  incidntnum  \
0  INVESTIGATIVE DETENTION   150494743   

                                            location pddistrict  \
0  {u'type': u'Point', u'coordinates': [-122.4150...   SOUTHERN   

             pdid resolution   time           x         y  
0  15049474364085       NONE  15:14 -122.415003  37.77265  


In [12]:
data_df['day'] = data_df.date.apply(lambda x: convert_date_to_day(x))
data_df['month'] = data_df.date.apply(lambda x: convert_date_to_month(x))
# data_df['year'] = data_df.date.apply(lambda x: convert_date_to_year(x))
data_df['hour'] = data_df.time.apply(lambda x: convert_time_to_hour(x))
data_df = data_df.sort_values(by=['date','time'])


In [166]:
# df = data_df.drop(['incidntnum','pdid','resolution','x','y', 'date', 'time', 'descript'], axis =1)
df = data_df.drop(['incidntnum','pdid','resolution','date', 'time', 'descript', 'location'], axis =1)
# df = df.drop(df.columns[[0]], axis =1)

for col in df.columns:
    print(col)

address
category
dayofweek
pddistrict
x
y
day
month
hour


In [167]:
# df.sort_values(by=['date'])
df['category'] = df.category.apply(lambda x: x.lower())
df['dayofweek'] = df.dayofweek.apply(lambda x: x.lower())
df['address'] = df.address.apply(lambda x: x.lower())
df['pddistrict'] = df.pddistrict.apply(lambda x: x.lower())

# Unique Crime Categories

In [16]:
uniqe_crime = {}
i = 0
for index,row in df.iterrows():
    if row['category'] not in uniqe_crime.keys():
        uniqe_crime[row['category']] = i
        i = i+1
    

In [17]:
print(len(uniqe_crime))
print(uniqe_crime)

39
{'weapon laws': 28, 'recovered vehicle': 37, 'secondary codes': 6, 'warrants': 15, 'prostitution': 10, 'drug/narcotic': 20, 'embezzlement': 9, 'trespass': 23, 'bribery': 32, 'driving under the influence': 22, 'non-criminal': 8, 'sex offenses, non forcible': 24, 'disorderly conduct': 18, 'other offenses': 5, 'runaway': 29, 'suicide': 33, 'liquor laws': 31, 'sex offenses, forcible': 7, 'vehicle theft': 17, 'vandalism': 13, 'gambling': 34, 'pornography/obscene mat': 36, 'kidnapping': 26, 'family offenses': 35, 'assault': 2, 'larceny/theft': 1, 'burglary': 4, 'missing person': 16, 'forgery/counterfeiting': 0, 'extortion': 14, 'fraud': 11, 'arson': 30, 'trea': 38, 'bad checks': 25, 'loitering': 12, 'stolen property': 27, 'robbery': 21, 'suspicious occ': 3, 'drunkenness': 19}


In [168]:
# Define Index Crimes (More Serious)
index_crimes = ['ROBBERY', 'BURGLARY', 'LARCENY/THEFT', 'ASSAULT', 'ARSON', 'SEX OFFENSES, FORCIBLE', 'SECONDARY CODES',  'RECOVERED VEHICLE']
print(len(index_crimes))

# Define Non-Index Crimes (Less Serious)
non_index_crimes = ['OTHER OFFENSES', 'VEHICLE THEFT', 'NON-CRIMINAL', 'SUSPICIOUS OCC', 'FRAUD', 'FORGERY/COUNTERFEITING', 'WARRANTS', 'VANDALISM', 'MISSING PERSON', 'DISORDERLY CONDUCT', 'TRESPASS', 'WEAPON LAWS', 'DRUG/NARCOTIC', 'STOLEN PROPERTY', 'DRUNKENNESS', 'EMBEZZLEMENT', 'LOITERING', 'DRIVING UNDER THE INFLUENCE', 'PROSTITUTION', 'LIQUOR LAWS', 'EXTORTION', 'RUNAWAY', 'SUICIDE', 'BAD CHECKS', 'KIDNAPPING', 'FAMILY OFFENSES', 'BRIBERY', 'GAMBLING', 'SEX OFFENSES, NON FORCIBLE', 'PORNOGRAPHY/OBSCENE MAT', 'TREA']
len(non_index_crimes)

8


31

# Find Severity

In [23]:
severity = []
for index, row in df.iterrows():
    if row['category'].upper() in index_crimes:
        severity.append('INDEX')
#         print(row['category'], ": Index Crime (More Serious)")
    else:
        severity.append('NONINDEX')
#         print(row['category'], " : None Index Crime (More Serious)")

In [24]:
print(len(severity))
print(df.shape)

2195019
(2195019, 9)


In [25]:
df['severity'] = severity

In [26]:
print(df['category'][:10])
print(df['severity'][:10])

1064904    forgery/counterfeiting
1073303             larceny/theft
1081823    forgery/counterfeiting
1142078                   assault
1175782            suspicious occ
1182298                  burglary
1194643                   assault
1198961            suspicious occ
1202350            other offenses
1203962           secondary codes
Name: category, dtype: object
1064904    NONINDEX
1073303       INDEX
1081823    NONINDEX
1142078       INDEX
1175782    NONINDEX
1182298       INDEX
1194643       INDEX
1198961    NONINDEX
1202350    NONINDEX
1203962       INDEX
Name: severity, dtype: object


In [169]:
print(df.shape)
print(sev_df.shape)

(2195019, 9)
(2195019, 9)


In [170]:
series = sev_df.groupby('pddistrict').apply(list)
dictionary = {}


In [171]:
city_target = sev_df.get('severity').tolist()
unique_target=set(city_target)
print(len(unique_target))
print(len(city_target))

2
2195019


In [172]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
lnc = LabelEncoder()
enc = OneHotEncoder()

In [174]:
def perform_label_encoding(temp_df):
    temp = lnc.fit_transform(temp_df.dayofweek.iloc[:].values)
    temp_df['dayofweek'] = temp
    temp = lnc.fit_transform(temp_df.address.iloc[:].values)
    temp_df['address'] = temp
    temp = lnc.fit_transform(temp_df.category.iloc[:].values)
    temp_df['category'] = temp
    return temp_df

In [177]:
df = perform_label_encoding(df)
print(type(df))
print(df.shape)
print(df[:5])
# perform_label_encoding(sev_df)

<class 'pandas.core.frame.DataFrame'>
(2195019, 9)
         address  category  dayofweek pddistrict           x          y day  \
1064904     3923        12          6    bayview -122.409221  37.731393  01   
1073303    12720        16          6   southern -122.403405  37.775421  01   
1081823     7502        12          6    mission -122.414683  37.758077  01   
1142078     1927         1          6    bayview -122.394538  37.734692  01   
1175782    12670        32          6   richmond -122.492762  37.773097  01   

        month hour  
1064904    01   00  
1073303    01   00  
1081823    01   00  
1142078    01   00  
1175782    01   00  


In [178]:
for s in series.keys():
    print(str(s))
    mask = df['pddistrict'] == str(s)
#     print(df[mask])
    dictionary[str(s)] = df[mask]

bayview
central
ingleside
mission
northern
park
richmond
southern
taraval
tenderloin


In [179]:
# for s in series.keys():
#     print(str(s))
#     mask = sev_df['pddistrict'] == str(s)
# #     print(df[mask])
#     dictionary[str(s)] = sev_df[mask]

In [180]:
print(type(dictionary['central']))
print(dictionary.keys())

<class 'pandas.core.frame.DataFrame'>
['central', 'northern', 'ingleside', 'park', 'mission', 'tenderloin', 'richmond', 'taraval', 'bayview', 'southern']


In [181]:
# def fetchDistrictData(district):
#     target = (dictionary.get(district))['severity']
#     tempdf = (dictionary.get(district)).drop(['pddistrict', 'severity', 'category'], axis =1)
#     return tempdf, target

In [184]:
def fetchDistrictData(district):
    target = (dictionary.get(district))['category']
    tempdf = (dictionary.get(district)).drop(['pddistrict', 'category','severity'], axis =1)
    return tempdf, target

In [185]:
district_bayview, district_bayview_target = fetchDistrictData('bayview')
print(len(district_bayview_target))
print(district_bayview.shape)

219306
(219306, 7)


In [186]:
# print(district_bayview.shape)
# print(district_bayview[:5])
print(len(set(district_bayview_target)))

39


In [100]:
from scipy.sparse import csr_matrix

def dataframetoCSRmatrix(df):
    nrows = len(df)
    nc = len(df.columns)
    idx = {}
    tid = 0
    nnz = nc * nrows
    cols= df.columns
    for col in cols:
        df[col] = df[col].apply(str)
        for name in df[col].unique():
            idx[col+name] = tid
            tid += 1
   
    ncols = len(idx)
   
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.int)
    ptr = np.zeros(nrows+1, dtype=np.int)
   
    i=0
    n=0
   
    for index,row in df.iterrows():
        for j,col in enumerate(cols):
            ind[j+n] = idx[col+row[col]]
            val[j+n] = 1
        ptr[i+1] = ptr[i] + nc
        n += nc
        i += 1
   
    mat = csr_matrix((val,ind,ptr),  dtype=np.int)
    mat.sort_indices()  
   
    return mat

In [101]:
mat_bayview = dataframetoCSRmatrix(district_bayview)
print("Shape of CSR Matrix -", mat_bayview.shape)

('Shape of CSR Matrix -', (219306, 12473))


In [102]:
print(type(mat_bayview))
print(mat_bayview.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(219306, 12473)


In [187]:
from sklearn.model_selection import train_test_split


In [188]:
def fetchTrainTestData(district_df, district_target):
    X_train, X_test, y_train, y_test = train_test_split(district_df, district_target, test_size=0.10, random_state=42)
    return X_train, X_test, y_train, y_test


In [189]:
# X_train, X_test, y_train, y_test = fetchTrainTestData(mat_bayview, district_bayview_target)
X_train, X_test, y_train, y_test = fetchTrainTestData(district_bayview, district_bayview_target)

In [190]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

(197375, 7)
(21931, 7)
(197375,)
(21931,)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [191]:
# X_train_list = X_train.values.tolist()
# X_test_list = X_test.values.tolist()

In [193]:
print(X_train[:10])
print(X_test[:10])

         address  dayofweek           x          y day month hour
1648680     1126          1 -122.379425  37.729499  14    04   19
1120372    18728          4 -122.386421  37.729545  17    08   09
451434     10404          4 -122.398512  37.767501  07    08   20
484476      5042          4 -122.392563  37.736874  24    07   18
2121678    12425          3 -122.373376  37.729769  15    08   09
503544      3807          0 -122.394144  37.722519  21    11   13
2101602    10032          4 -122.388004  37.755390  19    08   23
792491      4458          0 -122.389391  37.734121  14    10   13
547890     10932          4 -122.390136  37.736804  29    01   09
2079206     4162          0 -122.394439  37.736444  21    05   21
         address  dayofweek           x          y day month hour
703830      7559          1 -122.404952  37.730738  01    02   13
1521337    13248          1 -122.401962  37.757745  29    06   18
404752     11232          4 -122.401059  37.769775  28    03   23
155732    

In [194]:
def classify(clf, X_train, Y_train, X_test):
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, Y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)
    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    print(type(pred))
    return pred

In [195]:
# from sklearn.preprocessing import StandardScaler
# X_train_std = StandardScaler().fit_transform(X_train.values.tolist())
# X_test_std = StandardScaler().fit_transform(X_train.values.tolist())

In [196]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [201]:
# pred = classify(tree.DecisionTreeClassifier(), X_train, y_train, X_test)
# pred = classify(RandomForestClassifier(), X_train, y_train, X_test)
# pred = classify(GaussianNB(), X_train, y_train, X_test)
# pred = classify(KNeighborsClassifier(), X_train, y_train, X_test)
pred = classify(AdaBoostClassifier(), X_train, y_train, X_test)

Training: 
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
train time: 15.808s
test time:  1.073s
<type 'numpy.ndarray'>


In [202]:
print(pred)

[21 16 16 ..., 21 16 21]


In [203]:
from sklearn.metrics import f1_score

In [206]:
f1_score(y_test, pred, average='macro') 

ValueError: Target is multiclass but average='binary'. Please choose another average setting.